check here

[https://github.com/anujshah1003/own_data_cnn_implementation_keras/blob/master/custom_data_cnn.py]

[https://www.youtube.com/watch?v=NUuMg5m42-g]

In [ ]:
%matplotlib inline
from helpers import *

# import 'Sequential' is a linear stack of neural network layers. Will be used to build the feed-forward CNN
from keras.models import Sequential 
# import the "core" layers from Keras (these are the most common layers)
from keras.layers import Dense, Dropout, Activation, Flatten
# import the convolutional layers that will help us efficiently train on image data
from keras.layers import Conv2D, MaxPooling2D
# these utilities will help us transform our data
from keras.utils import np_utils

from keras import backend as K

%load_ext autoreload
%autoreload 2

#### Overview
The goal here is to use the CNN to reduce the size of the input image to obtain a "discretized" image of shape (W/16, H/16). Every entry of this image is related to a patch in the input image. This obtained image is compared by the CNN with the groundtruth, after discretizing by "discretizing" it patch-wise.

### - Load data

In [ ]:
# Loaded a set of images
n = 20

imgs, gt_imgs = load_images(n)

imgs[0].shape, gt_imgs[0].shape

### - For now avoid cross validation, just split the datasest in test and train. 

In [ ]:
train_ratio = 0.8
train, test = split_train_test(imgs, gt_imgs, train_ratio=train_ratio, seed=1)
train.imgs.shape, train.gt_imgs.shape, test.imgs.shape, test.gt_imgs.shape 

### - Generate the inpus and the outputs
We reshape each input to fulfill the requirements of the tensorflow library. 

In [ ]:
train.gt_imgs.shape

In [ ]:
asd = gt_imgs_to_Y(train.gt_imgs, 8)
asd.shape

In [ ]:
def gt_imgs_to_Y(gt_imgs, patch_width=8):
    gt_matr = np.array([img_crop_matr(gt_img, patch_width) for gt_img in gt_imgs])
    Y = np.zeros((gt_matr.shape[0], gt_matr.shape[1], gt_matr.shape[2])) # keep width and heigth but convert the patch to a class
    for im in range(Y.shape[0]):
        for i in range(Y.shape[1]):
            for j in range(Y.shape[2]):
                Y[im, i, j] = value_to_class(np.mean(gt_matr[im, i, j]))
    return np_utils.to_categorical(Y, 2)
    
train.X = train.imgs
train.Y = gt_imgs_to_Y(train.gt_imgs, 8)

test.X = test.imgs
test.Y = gt_imgs_to_Y(test.gt_imgs, 8)
train.X.shape, train.Y.shape, test.X.shape, test.Y.shape

Recap on **train** (and test) object:
- **train.imgs**: (expanded) images of the training set. **shape**=(#train images, 400, 400, 3)
- **train.gt_imgs**: respective groundtruth images. **shape**=(#train images, 400, 400)
- **train.X**: inputs extracted from train.imgs. **shape**=(#total windows, 72, 72, 3)
- **train.Y**: outputs extracted from train.gt_imgs. **shape**=(#total windows, 2)

### - Build the CNN model

How to build the cnn model:
- **Convolutional layers**: Conv1D or **Conv2D** (1D: covolution over a single spatial dimension)
    - **filters**: 

In [ ]:
nclasses = 2
model = Sequential()

# padding="same" => output width/heigth = input width/heigth

# layer 1
model.add(
    Conv2D(32, (11, 11), 
           activation='relu', 
           padding="same", 
           input_shape=train.X[0].shape)) # variable size input?
model.add(Dropout(0.25)) 

# layer 2
model.add(
    Conv2D(48, (5, 5),
           strides=(2, 2),
           activation='relu',
           padding="same"
          ))

# later 3
model.add(
    Conv2D(48, (5, 5), 
           activation='relu', 
           padding="same")) # variable size input
model.add(Dropout(0.25)) 

# layer 4
model.add(
    Conv2D(48, (5, 5),
           strides=(2, 2),
           activation='relu',
           padding="same"))

# layer 5
model.add(
    Conv2D(48, (5, 5), 
           activation='relu', 
           padding="same")) # variable size input
model.add(Dropout(0.25)) 

# layer 6
model.add(
    Conv2D(64, (5, 5), 
           strides=(2, 2),
           activation='relu', 
           padding="same")) # variable size input

# layer 7
model.add(
    Conv2D(64, (5, 5), 
           activation='relu', 
           padding="same")) # variable size input
model.add(Dropout(0.25)) 

# layer 8
model.add(
    Conv2D(2, (5, 5), 
           activation='relu', 
           padding="same")) # variable size input


model.add(Activation(K.softmax))

train.X[0].shape, model.output_shape

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

### - Fit the model on the train data

In [ ]:
train.X.shape, train.Y.shape

In [ ]:
# either validation_data=(test.X, test.Y) or validation_split=0.2
hist=model.fit(train.X, train.Y, batch_size=32, epochs=2, verbose=1)

In [ ]:
# visualizing losses and accuracy
train_loss=hist.history['loss']
val_loss=hist.history['val_loss']
train_acc=hist.history['acc']
val_acc=hist.history['val_acc']
xc=range(num_epoch)

plt.figure(1,figsize=(7,5))
plt.plot(xc,train_loss)
plt.plot(xc,val_loss)
plt.xlabel('num of Epochs')
plt.ylabel('loss')
plt.title('train_loss vs val_loss')
plt.grid(True)
plt.legend(['train','val'])
#print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])

plt.figure(2,figsize=(7,5))
plt.plot(xc,train_acc)
plt.plot(xc,val_acc)
plt.xlabel('num of Epochs')
plt.ylabel('accuracy')
plt.title('train_acc vs val_acc')
plt.grid(True)
plt.legend(['train','val'],loc=4)
#print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])

In [ ]:
# Visualizing the intermediate layer

#
def get_featuremaps(model, layer_idx, X_batch):
#     visualize layer n layer_idx
    get_activations = K.function([model.layers[0].input, K.learning_phase()],[model.layers[layer_idx].output,])
    activations = get_activations([X_batch,0])
    return activations

layer_num=3
filter_num=0

activations = get_featuremaps(model, int(layer_num),test_image)

print (np.shape(activations))
feature_maps = activations[0][0]      
print (np.shape(feature_maps))

if K.image_dim_ordering()=='th':
    feature_maps=np.rollaxis((np.rollaxis(feature_maps,2,0)),2,0)
print (feature_maps.shape)

# show just one filter relative to the layer
fig=plt.figure(figsize=(16,16))
plt.imshow(feature_maps[:,:,filter_num],cmap='gray')
plt.savefig("featuremaps-layer-{}".format(layer_num) + "-filternum-{}".format(filter_num)+'.jpg')

# show all the filters
num_of_featuremaps=feature_maps.shape[2]
fig=plt.figure(figsize=(16,16))	
plt.title("featuremaps-layer-{}".format(layer_num))
subplot_num=int(np.ceil(np.sqrt(num_of_featuremaps)))
for i in range(int(num_of_featuremaps)):
    ax = fig.add_subplot(subplot_num, subplot_num, i+1)
    #ax.imshow(output_image[0,:,:,i],interpolation='nearest' ) #to see the first filter
    ax.imshow(feature_maps[:,:,i],cmap='gray')
    plt.xticks([])
    plt.yticks([])
    plt.tight_layout()
plt.show()
fig.savefig("featuremaps-layer-{}".format(layer_num) + '.jpg')
# you should be able to see that the cnn is keeping les information while the image goes through (keep only the
# important features)

### - Evaluate the model on the test data

In [ ]:
# score = model.evaluate(test.X, test.Y, verbose=1)
# score

In [ ]:
test.X.shape

In [ ]:
Z = model.predict_classes(test.X, verbose=1)
# stats(Z, test.Y[:, 1])
Z.shape, test.Y.shape

In [ ]:
i = 0
display_ith_prediction(test, Z, i, window_width)

In [ ]:
# Printing the confusion matrix
from sklearn.metrics import classification_report,confusion_matrix
import itertools

Y_pred = model.predict(X_test)
print(Y_pred)
y_pred = np.argmax(Y_pred, axis=1)
print(y_pred)
#y_pred = model.predict_classes(X_test)
#print(y_pred)
target_names = ['class 0(cats)', 'class 1(Dogs)', 'class 2(Horses)','class 3(Humans)']

# print precision, recall, F1 score
print(classification_report(np.argmax(y_test,axis=1), y_pred,target_names=target_names))

print(confusion_matrix(np.argmax(y_test,axis=1), y_pred))


# Plotting the confusion matrix
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Compute confusion matrix
cnf_matrix = (confusion_matrix(np.argmax(y_test,axis=1), y_pred))

np.set_printoptions(precision=2)

plt.figure()

# Plot non-normalized confusion matrix
plot_confusion_matrix(cnf_matrix, classes=target_names,
                      title='Confusion matrix')
#plt.figure()
# Plot normalized confusion matrix
#plot_confusion_matrix(cnf_matrix, classes=target_names, normalize=True,
#                      title='Normalized confusion matrix')
#plt.figure()
plt.show()

**problem**: it predicts always the same class [check here](https://stackoverflow.com/questions/41488279/neural-network-always-predicts-the-same-class)

### - Save/load model

In [ ]:
# Saving and loading model and weights
from keras.models import model_from_json
from keras.models import load_model

# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

model.save('model.hdf5')
loaded_model=load_model('model.hdf5')